runs g:profiler for 'omics gene set

using g:Profiler API

https://biit.cs.ut.ee/gprofiler/page/apis

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd


In [2]:
adj_pval_thresh = 0.05

In [3]:
pd.set_option('display.max_columns', None)

Need the file update the gene symbol, run [this notebook](1B-1[Omics_revamped]Gene_symbol_update.ipynb) for the data/omics_revamped_LLM_updated_genes_DF.tsv File

## Read in naming file

In [4]:
inputFile = 'data/omics_revamped_LLM_updated_genes_DF.tsv'
outputFile = 'data/omics_revamped_LLM_gprofiler_new_gene_name_DF.tsv' #save for the updated gene names


geneSeparator = " "

genesCol = 'updated GeneList' # 'GeneList'

In [5]:
genesets_LLM = pd.read_csv(inputFile, sep = "\t") 
print(genesets_LLM.shape)
genesets_LLM.head() 

(300, 9)


,Source,GeneSetID,GeneSetName,GeneList,updated GeneList,n_Genes,LLM Name,LLM Analysis,Score
0,NeST,Cluster1-10,Cluster1-10,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,12,Lipid Metabolism and Membrane Dynamics,"1. CTRL, or carboxyl-terminal esterase/lipase,...",0.85
1,NeST,Cluster1-11,Cluster1-11,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,12,Lipid Metabolism and Membrane Trafficking,1. LMF1 (Lipase Maturation Factor 1) is crucia...,0.85
2,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,Lipid Metabolism and Receptor-Mediated Endocyt...,"1. AMY2B (Amylase, alpha 2B) is an enzyme that...",0.88
3,NeST,Cluster1-13,Cluster1-13,ACO1 CDKAL1 CDX2 CIAO1 CITED2 FAM96A FAM96B FB...,ACO1 CDKAL1 CDX2 CIAO1 CITED2 CIAO2A CIAO2B FB...,16,Iron Metabolism and Regulation in Pancreatic F...,"1. ACO1, also known as aconitase 1, is a key e...",0.88
4,NeST,Cluster1-14,Cluster1-14,DTWD1 DTWD2 POLD2 POLD4 POLH POLI RAD18 REV1 R...,DTWD1 DTWD2 POLD2 POLD4 POLH POLI RAD18 REV1 R...,15,DNA Damage Tolerance and Repair Pathway,1. DTWD1 and DTWD2 are proteins containing DTW...,0.92


In [6]:
LLM_DF = genesets_LLM.copy()

In [7]:
nGeneSets = LLM_DF.shape[0]
nGeneSets

300

In [8]:
LLM_repeated_DF = LLM_DF[LLM_DF[['Source', 'GeneSetID', 'GeneSetName', 'GeneList']].duplicated() == True]
LLM_repeated_DF

,Source,GeneSetID,GeneSetName,GeneList,updated GeneList,n_Genes,LLM Name,LLM Analysis,Score


## Go through each system and run g:Profiler

In [9]:
# Each system will have multiple rows

In [14]:

import requests

def query_gprofiler(query_genes) -> pd.DataFrame:
    res_df = pd.DataFrame(columns=['Term', 'GO ID', 'Adjusted P-value', 'intersection_size',  'term_size', 'query_size', 'intersections'])
    r = requests.post(
        url='https://biit.cs.ut.ee/gprofiler/api/gost/profile/',
        json={
            'organism': 'hsapiens',
            'query': query_genes,
            'sources': ['GO:BP'], # query only GO BP
            'user_threshold': 0.05, 
            'significance_threshold_method': 'fdr', # using fdr for adjusted p 
            'all_results': True # get all results, not just significant ones
        }
        )
    result = r.json()['result']
    # Check if the result is not empty
    if result:
        # Extract relevant fields and append to the DataFrame
        for entry in result:
            # The elements in this list correspond to query genes and are in the same order as the genes
            intersection_genes = [
                query_genes[i] for i, intersect in enumerate(entry['intersections']) if intersect
            ]
            # print(entry['intersections'])
            newrow ={
                'Term': entry['name'],
                'GO ID': entry['native'],
                'Adjusted P-value': entry['p_value'],
                'intersection_size': entry['intersection_size'],
                'term_size': entry['term_size'],
                'query_size': entry['query_size'],
                'intersections': ','.join(intersection_genes) # Convert list to a space-separated string
            }
            res_df = pd.concat([res_df, pd.DataFrame([newrow])], ignore_index=True)
    else:
        raise ValueError('No results found for the query genes')
    return res_df.sort_values('Adjusted P-value', ascending=True).reset_index(drop=True)

# one example running case
# query_genes = LLM_DF['GeneList'][0].split(geneSeparator)
# print(query_genes )
# res_df = query_gprofiler(query_genes)
# res_df
genes = [
    "ACBD6", "ACRV1", "ACYP1", "C4orf48", "CAMTA1", "CCDC85B", "CTRL", "EPG5", 
    "GPR75", "HIST1H2AH", "HIST1H2AI", "HIST1H2BH", "HIST1H3F", "HIST1H4A", 
    "HIST1H4K", "HIST3H2A", "NRM", "NUDT3", "OGFOD2", "PRR22", "PUSL1", "QPCTL", 
    "RCCD1", "SNX22", "ST20", "SYCE1L", "TCTEX1D2", "TMSB15B", "TOX", "TOX2", 
    "ZNF236", "ZNF726"
]
query_gprofiler(genes)

,Term,GO ID,Adjusted P-value,intersection_size,term_size,query_size,intersections
0,peptidyl-pyroglutamic acid biosynthetic proces...,GO:0017186,0.105748,1,2,20,HIST3H2A
1,"CD8-positive, alpha-beta T cell lineage commit...",GO:0043375,0.105748,1,2,20,PUSL1
2,diphosphoinositol polyphosphate catabolic process,GO:0071544,0.105748,1,1,20,HIST1H2BH
3,inositol phosphate catabolic process,GO:0071545,0.105748,1,2,20,HIST1H2BH
4,cellular response to dsDNA,GO:1990786,0.105748,1,2,20,EPG5
...,...,...,...,...,...,...,...
440,negative regulation of macromolecule biosynthe...,GO:0010558,1.000000,1,2773,20,CCDC85B
441,cellular process,GO:0009987,1.000000,14,19708,20,"ACYP1,CAMTA1,CCDC85B,EPG5,GPR75,HIST1H2BH,HIST..."
442,gene expression,GO:0010467,1.000000,7,9214,20,"CAMTA1,CCDC85B,HIST1H4K,PUSL1,QPCTL,RCCD1,SNX22"
443,establishment of localization in cell,GO:0051649,1.000000,1,1972,20,EPG5


In [12]:
expandedColumns = list(LLM_DF.columns)
additionalCols = ['Term', 'GO ID', 'Adjusted P-value', 'intersection_size', 'term_size', 'query_size', 'intersections']
expandedColumns.extend(additionalCols)
expandedColumns

['Source',
 'GeneSetID',
 'GeneSetName',
 'GeneList',
 'updated GeneList',
 'n_Genes',
 'LLM Name',
 'LLM Analysis',
 'Score',
 'Term',
 'GO ID',
 'Adjusted P-value',
 'intersection_size',
 'term_size',
 'query_size',
 'intersections']

In [13]:
# initialize the expanded DataFrame
expanded_LLM_genes_DF = pd.DataFrame(columns=expandedColumns)

In [14]:
# Loop over all gene sets 
for geneSetInd in range(nGeneSets):
    print(geneSetInd)
    geneSetRow = LLM_DF.iloc[geneSetInd]
    geneSetGenes = geneSetRow[genesCol].split(geneSeparator)

    # Perform gProfiler enrichment analysis get the full table
    sigRes_DF = query_gprofiler(geneSetGenes)

    # thresholding

    sigRes_DF = sigRes_DF.sort_values('Adjusted P-value').reset_index(drop=True)
    
    if sigRes_DF.shape[0] == 0: # failed enrichment analysis
        
        print([geneSetInd, ' has not enirchment'])
        #continue # skip to next gene set
        sigRes_DF.loc[0, ['Term', 'GO ID', 'Adjusted P-value', 'intersection_size', 'term_size', 'query_size', 'intersections']] = 'NaN'
  
        
    else:

        if sigRes_DF.loc[0, 'Adjusted P-value'] > adj_pval_thresh: # nothing significant, return the first on the list
            sigRes_DF = sigRes_DF.loc[[0],:]
        else: # there are sig. terms, return all
            sigRes_DF = sigRes_DF[sigRes_DF['Adjusted P-value'] <=adj_pval_thresh]
        
    # Doing what should be a simple colbind 
    geneSetRow_DF = geneSetRow.to_frame().T
    geneSetRow_repeated_DF = geneSetRow_DF.loc[geneSetRow_DF.index.repeat(sigRes_DF.shape[0])]
    geneSetRow_repeated_DF = geneSetRow_repeated_DF.reset_index()
    
    # system_Row_sigRes = systemRow_repeated_DF.join(sigRes_DF)
    geneSetRowRow_sigRes = pd.concat([geneSetRow_repeated_DF, sigRes_DF],axis=1)
    
    # Doing what should be a simple  rowbind
    if expanded_LLM_genes_DF.shape[0] == 0:
        expanded_LLM_genes_DF = geneSetRowRow_sigRes[expandedColumns]
    else:
        expanded_LLM_genes_DF =  pd.concat([expanded_LLM_genes_DF, geneSetRowRow_sigRes[expandedColumns]],axis=0)
    
    
    expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t", index=False) # Save to not loose work

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
expanded_LLM_genes_DF.loc[expanded_LLM_genes_DF["GeneSetID"] == 'Cluster2-191']

,Source,GeneSetID,GeneSetName,GeneList,updated GeneList,n_Genes,LLM Name,LLM Analysis,Score,Term,GO ID,Adjusted P-value,intersection_size,term_size,query_size,intersections
0,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,non-canonical Wnt signaling pathway,GO:0035567,1.708941e-27,14,70,22,"CELSR1,CELSR2,CELSR3,FZD1,FZD2,FZD3,FZD5,FZD6,..."
1,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,cell-cell signaling by wnt,GO:0198738,4.033949e-20,16,454,22,"APC2,CELSR1,CELSR2,CELSR3,FUZ,FZD1,FZD2,FZD3,F..."
2,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,Wnt signaling pathway,GO:0016055,4.033949e-20,16,452,22,"APC2,CELSR1,CELSR2,CELSR3,FUZ,FZD1,FZD2,FZD3,F..."
3,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,"Wnt signaling pathway, planar cell polarity pa...",GO:0060071,3.728262e-19,10,50,22,"CELSR1,CELSR2,CELSR3,FZD1,FZD2,FZD3,FZD6,FZD7,..."
4,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,canonical Wnt signaling pathway,GO:0060070,8.454424e-15,12,305,22,"APC2,FUZ,FZD1,FZD2,FZD3,FZD5,FZD6,FZD7,FZD8,FZ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,negative regulation of mitochondrial outer mem...,GO:1901029,4.861368e-02,1,15,22,FZD9
339,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,nephron tubule epithelial cell differentiation,GO:0072160,4.861368e-02,1,15,22,FAT4
340,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,leukocyte activation,GO:0045321,4.861368e-02,4,946,22,"FZD5,FZD7,FZD8,FZD9"
341,NeST,Cluster2-191,Cluster2-191,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,APC2 CELSR1 CELSR2 CELSR3 FAT4 FREM2 FUZ FZD1 ...,22,Wnt/Planar Cell Polarity Signaling,"1. The proteins APC2, CELSR1, CELSR2, CELSR3, ...",0.92,midbrain dopaminergic neuron differentiation,GO:1904948,4.861368e-02,1,15,22,FZD1


In [16]:
# remove NAN values
expanded_LLM_genes_DF = expanded_LLM_genes_DF.dropna(subset=['Term'])

expanded_LLM_genes_DF.shape

(53452, 16)

In [17]:
expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t", index=False) 

In [18]:
#
use_update_name = pd.read_csv('data/omics_revamped_LLM_gprofiler_new_gene_name_DF.tsv', sep = "\t")
use_default_name = pd.read_csv('data/omics_revamped_LLM_gprofiler_default_gene_name_DF.tsv', sep = "\t")

use_update_name.equals(use_default_name)

False

In [19]:
# if the Terms are different
print(use_update_name['Term'].equals(use_default_name['Term']))
#if not, check the difference
diff = use_update_name[~use_update_name['Term'].isin(use_default_name['Term'])].index
print(len(diff))

False
128


In [20]:
use_update_name.loc[diff, ['GeneSetID', 'LLM Name', 'Term', 'Adjusted P-value', 'intersection_size', 'term_size','intersections']]

,GeneSetID,LLM Name,Term,Adjusted P-value,intersection_size,term_size,intersections
473,Cluster1-13,Iron Metabolism and Regulation in Pancreatic F...,protein maturation by [4Fe-4S] cluster transfer,0.000882,2,7,"CIAO2A,CIAO2B"
1401,Cluster1-3,Mitochondrial Biogenesis and Function,mitochondrial RNA 5'-end processing,0.002310,2,4,"KHDRBS1,PFDN5"
1465,Cluster1-3,Mitochondrial Biogenesis and Function,tRNA 5'-leader removal,0.017892,2,13,"KHDRBS1,SRSF10"
1477,Cluster1-3,Mitochondrial Biogenesis and Function,tRNA 5'-end processing,0.024982,2,16,"KHDRBS1,SRSF10"
1997,Cluster1-9,Protein Quality Control and Intracellular Traf...,arsenite transport,0.019149,1,1,GET3
...,...,...,...,...,...,...,...
53054,Ebolavirus(ZEBOV)_6day_PBMCs_rNAPc2treated,System of unrelated proteins,regulation of extrathymic T cell differentiation,0.164711,1,2,FOXD2
53097,HCoV-EMC2012_24Hour,System of unrelated proteins,regulation of ribosomal subunit export from nu...,0.136722,1,1,RIOK2
53098,HHV-8_72Hour,System of unrelated proteins,epithelial fluid transport,0.313999,1,12,AHCYL1
53107,SARS-CoV MA15_Day1,System of unrelated proteins,protein N-linked glycosylation via arginine,0.223802,1,1,KRTCAP2


In [21]:
use_default_name.loc[diff, ['GeneSetID', 'LLM Name', 'Term', 'Adjusted P-value', 'intersection_size', 'term_size','intersections']]

,GeneSetID,LLM Name,Term,Adjusted P-value,intersection_size,term_size,intersections
473,Cluster1-13,Iron Metabolism and Regulation in Pancreatic F...,regulation of metabolic process,0.000551,13,7158,"ACO1,CDX2,CIAO1,CITED2,FAM96A,FAM96B,FBXL5,IRE..."
1401,Cluster1-3,Mitochondrial Biogenesis and Function,positive regulation of cellular catabolic process,0.001802,8,332,"ERAL1,GFM2,HIST1H1B,NOB1,RRP1B,SLC25A5,SSB,TXN2"
1465,Cluster1-3,Mitochondrial Biogenesis and Function,respiratory chain complex III assembly,0.014871,2,12,"BCS1L,TPR"
1477,Cluster1-3,Mitochondrial Biogenesis and Function,regulation of metabolic process,0.017982,44,7158,"AATF,ABT1,AURKAIP1,COQ3,COX17,CWC22,DDX49,DIEX..."
1997,Cluster1-9,Protein Quality Control and Intracellular Traf...,stress granule assembly,0.016232,2,31,"DNAJC14,TRAPPC11"
...,...,...,...,...,...,...,...
53054,BRD-A42737819_-666_MCF7_6.0_h_10.0_um,System of unrelated proteins,anatomical structure development,0.000221,37,5899,"1060P11.3,AKR1B1,ANXA5,ARG2,CKAP4,COCH,CXCL8,D..."
53097,BRD-A42737819_-666_MCF7_6.0_h_10.0_um,System of unrelated proteins,regulation of apoptotic process,0.004777,14,1463,"1060P11.3,AKR1B1,ALDH1A2,ANXA5,ARG2,BTG1,PEG3,..."
53098,BRD-A42737819_-666_MCF7_6.0_h_10.0_um,System of unrelated proteins,cellular response to lipid,0.004777,9,609,"AKR1B1,CCND2,COCH,ITM2A,PIP,TRIM29,DNAJA3,KAZN..."
53107,BRD-A42737819_-666_MCF7_6.0_h_10.0_um,System of unrelated proteins,innate immune response,0.005656,11,965,"ANXA5,CCND2,DPYSL2,FCGBP,PIP,TIPARP,COL3A1,HIS..."
